In [4]:
import requests
import pandas as pd

In [46]:
data = requests.request("GET", 'http://ergast.com/api/f1/drivers?limit=10000').text

In [48]:
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )